In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [187]:
import numpy as np
import re
import tensorflow as tf
import pickle

from tensorflow.keras.layers import Dense, GRU,LSTM, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
with open("BERT_embeds.pkl", "rb") as file_to_read:
    corpus_embeds = pickle.load(file_to_read)
    
corpus_embeds_np = np.asarray(corpus_embeds)
corpus_embeds_np.shape

(61200, 768)

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [176]:
with open("Classes_KMeans_3.pkl", "rb") as file_to_read:
    classes_KM = pickle.load(file_to_read)
    
classes_KM_np = np.asarray(classes_KM)
classes_KM_np.shape

(61200,)

In [92]:
total_lines = classes_KM_np.shape[0]
count_true = classes_KM_np.sum()
count_false = classes_KM_np.shape[0] - classes_KM_np.sum()
print(count_true, count_false, total_lines)

19387 41813 61200


In [93]:
X = corpus_embeds_np
Y = np.array([[1, 0]]*count_true + [[0, 1]]*count_false)
print(X.shape, Y.shape)

(61200, 768) (61200, 2)


In [97]:
for i in range (total_lines):
    if classes_KM_np[i] == 0:
        Y[i] = np.array([[1,0]])
    if classes_KM_np[i]==1:
        Y[i] = np.array([[0,1]])

In [177]:
classes_KM_np_dum = pd.get_dummies(classes_KM_np)
classes_KM_np_dum

,0,1,2,3,4,5
0,0,0,0,0,0,1
1,0,0,0,0,1,0
2,0,0,0,1,0,0
3,0,0,1,0,0,0
4,0,0,0,0,1,0
...,...,...,...,...,...,...
61195,0,1,0,0,0,0
61196,0,0,0,1,0,0
61197,0,1,0,0,0,0
61198,0,0,1,0,0,0


In [99]:
model_1 = Sequential()
model_1.add(Embedding(1000, 128, input_length = 768))
model_1.add(GRU(128, return_sequences=True))
model_1.add(GRU(64))
model_1.add(Dense(2, activation='softmax'))
model_1.summary()

model_1.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 768, 128)          128000    
                                                                 
 gru_4 (GRU)                 (None, 768, 128)          99072     
                                                                 
 gru_5 (GRU)                 (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 2)                 130       
                                                                 
Total params: 264,450
Trainable params: 264,450
Non-trainable params: 0
_________________________________________________________________


In [27]:
model = Sequential()
model.add(Embedding(1000, 128, input_length = 768))
model.add(GRU(128, return_sequences=True))
model.add(GRU(64))
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))




Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 768, 128)          128000    
                                                                 
 gru_2 (GRU)                 (None, 768, 128)          99072     
                                                                 
 gru_3 (GRU)                 (None, 64)                37248     
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 264,450
Trainable params: 264,450
Non-trainable params: 0
_________________________________________________________________


In [100]:
history = model_1.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
1913/1913 [==============================] - 81s 42ms/step - loss: 0.3187 - accuracy: 0.8608
Epoch 2/50
1913/1913 [==============================] - 79s 41ms/step - loss: 0.1838 - accuracy: 0.9376
Epoch 3/50
1913/1913 [==============================] - 80s 42ms/step - loss: 0.1700 - accuracy: 0.9426
Epoch 4/50
1913/1913 [==============================] - 80s 42ms/step - loss: 0.1562 - accuracy: 0.9464
Epoch 5/50
1913/1913 [==============================] - 81s 42ms/step - loss: 0.1385 - accuracy: 0.9517
Epoch 6/50
1913/1913 [==============================] - 80s 42ms/step - loss: 0.0772 - accuracy: 0.9728
Epoch 7/50
1913/1913 [==============================] - 80s 42ms/step - loss: 0.0506 - accuracy: 0.9816
Epoch 8/50
1913/1913 [==============================] - 80s 42ms/step - loss: 0.0481 - accuracy: 0.9815
Epoch 9/50
1913/1913 [==============================] - 80s 42ms/step - loss: 0.0451 - accuracy: 0.9825
Epoch 10/50
1913/1913 [==============================] - 80s 42m

In [29]:
history_1 = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
1913/1913 [==============================] - 81s 42ms/step - loss: 0.4345 - accuracy: 0.7893
Epoch 2/50
1913/1913 [==============================] - 79s 41ms/step - loss: 0.2941 - accuracy: 0.8813
Epoch 3/50
1913/1913 [==============================] - 80s 42ms/step - loss: 0.2639 - accuracy: 0.8906
Epoch 4/50
1913/1913 [==============================] - 80s 42ms/step - loss: 0.2532 - accuracy: 0.8945
Epoch 5/50
1913/1913 [==============================] - 80s 42ms/step - loss: 0.2478 - accuracy: 0.8954
Epoch 6/50
1913/1913 [==============================] - 80s 42ms/step - loss: 0.2429 - accuracy: 0.8975
Epoch 7/50
1913/1913 [==============================] - 80s 42ms/step - loss: 0.2874 - accuracy: 0.8811
Epoch 8/50
1913/1913 [==============================] - 80s 42ms/step - loss: 0.2376 - accuracy: 0.8987
Epoch 9/50
1913/1913 [==============================] - 80s 42ms/step - loss: 0.2354 - accuracy: 0.8999
Epoch 10/50
1913/1913 [==============================] - 80s 42m

In [59]:
import os
import pandas as pd
import numpy as np
import pickle
import warnings
from multiprocessing import Process, Lock
import tensorflow_hub as hub
import tensorflow_text
import torch

In [60]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-conversational")
model_bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-conversational")

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [117]:
def bert_transf(text, batch_l = 2):
    train_encodings = tokenizer(text, truncation=True, padding='max_length', max_length=256)
    input_ids = torch.tensor(train_encodings['input_ids'])
    BATCH_LENGTH = batch_l
    embeddings = np.empty((0,768))
    device = torch.device("cuda")
    device2 = torch.device("cpu")
    model_bert.to('cuda')
    with torch.no_grad():
        for batch_num in range(int(len(input_ids) / BATCH_LENGTH)):
            try:
                batch = input_ids[batch_num * BATCH_LENGTH : (batch_num + 1) * BATCH_LENGTH].cuda()
                results = model_bert(batch)
                head_output = results[0][:,0,:].cpu().detach().numpy()
                embeddings = np.concatenate((embeddings, head_output))
                print(f"processed batch {batch_num}")
            except Exception:
                pass
    res = model_1.predict(embeddings)
    print(res, np.argmax(res), sep='\n')

In [118]:
t = ['павел любовь значит наша ограбить ключи готов и у меня одного ключа не хватает прав да прибор листочек не дали не медали минутку сейчас я подниму перешлю вам хорошо когда рассказываешь хорошо ']
emb = bert_transf(t, batch_l = 1)


processed batch 0
[[9.9999654e-01 3.4453303e-06]]
0


In [171]:
res = model_1.predict(X[0:1])
print(res, sep='\n')

[[9.9991965e-01 8.0342274e-05]]


In [164]:
classes_KM_np[2]

0

In [165]:
if res[0][0]>res[0][1]:
    print('0')
else:
    print('1')

1


In [178]:
model_2 = Sequential()
model_2.add(Embedding(1000, 128, input_length = 768))
model_2.add(GRU(128, return_sequences=True))
model_2.add(GRU(64))
model_2.add(Dense(6, activation='softmax'))
model_2.summary()

model_2.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 768, 128)          128000    
                                                                 
 gru_6 (GRU)                 (None, 768, 128)          99072     
                                                                 
 gru_7 (GRU)                 (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 6)                 390       
                                                                 
Total params: 264,710
Trainable params: 264,710
Non-trainable params: 0
_________________________________________________________________


In [180]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, classes_KM_np_dum, test_size=0.1, random_state=42)

In [182]:
history_3 = model_2.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test,y_test))

Epoch 1/50
1722/1722 [==============================] - 89s 46ms/step - loss: 1.4641 - accuracy: 0.4044 - val_loss: 1.2720 - val_accuracy: 0.4887
Epoch 2/50
1722/1722 [==============================] - 75s 43ms/step - loss: 1.1817 - accuracy: 0.5233 - val_loss: 1.0434 - val_accuracy: 0.5729
Epoch 3/50
1722/1722 [==============================] - 75s 44ms/step - loss: 1.0060 - accuracy: 0.5884 - val_loss: 0.9486 - val_accuracy: 0.6095
Epoch 4/50
1722/1722 [==============================] - 75s 44ms/step - loss: 0.9293 - accuracy: 0.6183 - val_loss: 0.8846 - val_accuracy: 0.6386
Epoch 5/50
1722/1722 [==============================] - 75s 44ms/step - loss: 0.8951 - accuracy: 0.6351 - val_loss: 0.8709 - val_accuracy: 0.6371
Epoch 6/50
1722/1722 [==============================] - 75s 44ms/step - loss: 0.8747 - accuracy: 0.6442 - val_loss: 0.8943 - val_accuracy: 0.6261
Epoch 7/50
1722/1722 [==============================] - 76s 44ms/step - loss: 0.8603 - accuracy: 0.6518 - val_loss: 0.8519 -

In [194]:
model_3 = Sequential()
model_3.add(Embedding(1000, 128, input_length = 768))
model_3.add(GRU(128, return_sequences=True))
model_3.add(GRU(64))
model_3.add(Dropout(0.8))
model_3.add(Dense(6, activation='softmax'))
model_3.summary()

model_3.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 768, 128)          128000    
                                                                 
 gru_16 (GRU)                (None, 768, 128)          99072     
                                                                 
 gru_17 (GRU)                (None, 64)                37248     
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 6)                 390       
                                                                 
Total params: 264,710
Trainable params: 264,710
Non-trainable params: 0
_________________________________________________________________


In [195]:
history_4 = model_3.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test,y_test))

Epoch 1/50
1722/1722 [==============================] - 78s 44ms/step - loss: 1.6322 - accuracy: 0.3103 - val_loss: 1.3965 - val_accuracy: 0.4735
Epoch 2/50
1722/1722 [==============================] - 75s 44ms/step - loss: 1.3908 - accuracy: 0.4574 - val_loss: 1.2791 - val_accuracy: 0.5046
Epoch 3/50
1722/1722 [==============================] - 76s 44ms/step - loss: 1.3140 - accuracy: 0.4800 - val_loss: 1.1797 - val_accuracy: 0.5224
Epoch 4/50
1722/1722 [==============================] - 75s 44ms/step - loss: 1.2396 - accuracy: 0.5086 - val_loss: 1.1025 - val_accuracy: 0.5642
Epoch 5/50
1722/1722 [==============================] - 76s 44ms/step - loss: 1.1800 - accuracy: 0.5318 - val_loss: 1.0542 - val_accuracy: 0.5804
Epoch 6/50
1722/1722 [==============================] - 76s 44ms/step - loss: 1.1283 - accuracy: 0.5596 - val_loss: 1.0151 - val_accuracy: 0.5967
Epoch 7/50
1722/1722 [==============================] - 76s 44ms/step - loss: 1.1040 - accuracy: 0.5752 - val_loss: 1.0065 -

In [190]:
model_4 = Sequential()
model_4.add(Embedding(1000, 128, input_length = 768))
model_4.add(LSTM(128, return_sequences=True))
model_4.add(LSTM(64))
model_4.add(Dense(6, activation='softmax'))
model_4.summary()

model_4.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 768, 128)          128000    
                                                                 
 lstm_2 (LSTM)               (None, 768, 128)          131584    
                                                                 
 lstm_3 (LSTM)               (None, 64)                49408     
                                                                 
 dense_7 (Dense)             (None, 6)                 390       
                                                                 
Total params: 309,382
Trainable params: 309,382
Non-trainable params: 0
_________________________________________________________________


In [191]:
history_5 = model_4.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test,y_test))

Epoch 1/50
1722/1722 [==============================] - 79s 45ms/step - loss: 1.4448 - accuracy: 0.4044 - val_loss: 1.2200 - val_accuracy: 0.5106
Epoch 2/50
1722/1722 [==============================] - 76s 44ms/step - loss: 1.2139 - accuracy: 0.5119 - val_loss: 1.1175 - val_accuracy: 0.5505
Epoch 3/50
1722/1722 [==============================] - 76s 44ms/step - loss: 1.1466 - accuracy: 0.5373 - val_loss: 1.0447 - val_accuracy: 0.5784
Epoch 4/50
1722/1722 [==============================] - 76s 44ms/step - loss: 1.0135 - accuracy: 0.5888 - val_loss: 0.9598 - val_accuracy: 0.6152
Epoch 5/50
1722/1722 [==============================] - 77s 45ms/step - loss: 0.9628 - accuracy: 0.6097 - val_loss: 0.9353 - val_accuracy: 0.6266
Epoch 6/50
1722/1722 [==============================] - 77s 45ms/step - loss: 0.9516 - accuracy: 0.6141 - val_loss: 0.9205 - val_accuracy: 0.6325
Epoch 7/50
1722/1722 [==============================] - 77s 45ms/step - loss: 0.9475 - accuracy: 0.6132 - val_loss: 0.8893 -

In [198]:
with open("Classes_KMeans_4.pkl", "rb") as file_to_read:
    classes_KM_1 = pickle.load(file_to_read)
    
classes_KM_np_1 = np.asarray(classes_KM)
classes_KM_np_dum_1 = pd.get_dummies(classes_KM_np_1)
classes_KM_np_dum_1

,0,1,2,3,4,5
0,0,0,0,0,0,1
1,0,0,0,0,1,0
2,0,0,0,1,0,0
3,0,0,1,0,0,0
4,0,0,0,0,1,0
...,...,...,...,...,...,...
61195,0,1,0,0,0,0
61196,0,0,0,1,0,0
61197,0,1,0,0,0,0
61198,0,0,1,0,0,0


In [199]:
model_5 = Sequential()
model_5.add(Embedding(1000, 128, input_length = 768))
model_5.add(GRU(128, return_sequences=True))
model_5.add(GRU(64))
model_5.add(Dense(6, activation='softmax'))
model_5.summary()

model_5.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 768, 128)          128000    
                                                                 
 gru_18 (GRU)                (None, 768, 128)          99072     
                                                                 
 gru_19 (GRU)                (None, 64)                37248     
                                                                 
 dense_11 (Dense)            (None, 6)                 390       
                                                                 
Total params: 264,710
Trainable params: 264,710
Non-trainable params: 0
_________________________________________________________________


In [200]:
from sklearn.model_selection import train_test_split
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, classes_KM_np_dum_1, test_size=0.1, random_state=42)

In [201]:
history_6 = model_5.fit(X_train_1, y_train_1, batch_size=32, epochs=50, validation_data=(X_test_1,y_test_1))

Epoch 1/50
1722/1722 [==============================] - 78s 44ms/step - loss: 1.4790 - accuracy: 0.3968 - val_loss: 1.2444 - val_accuracy: 0.5049
Epoch 2/50
1722/1722 [==============================] - 75s 43ms/step - loss: 1.1619 - accuracy: 0.5341 - val_loss: 1.0457 - val_accuracy: 0.5779
Epoch 3/50
1722/1722 [==============================] - 75s 44ms/step - loss: 1.0148 - accuracy: 0.5889 - val_loss: 0.9553 - val_accuracy: 0.6080
Epoch 4/50
1722/1722 [==============================] - 75s 44ms/step - loss: 0.9426 - accuracy: 0.6131 - val_loss: 0.8985 - val_accuracy: 0.6413
Epoch 5/50
1722/1722 [==============================] - 76s 44ms/step - loss: 0.9163 - accuracy: 0.6247 - val_loss: 0.8786 - val_accuracy: 0.6467
Epoch 6/50
1722/1722 [==============================] - 76s 44ms/step - loss: 0.8980 - accuracy: 0.6323 - val_loss: 0.8761 - val_accuracy: 0.6451
Epoch 7/50
1722/1722 [==============================] - 76s 44ms/step - loss: 0.8810 - accuracy: 0.6403 - val_loss: 0.8547 -

In [202]:
model_anom = Sequential()
model_anom.add(Embedding(1000, 128, input_length = 768))
model_anom.add(GRU(128, return_sequences=True))
model_anom.add(GRU(64))
model_anom.add(Dense(2, activation='softmax'))
model_anom.summary()

model_anom.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 768, 128)          128000    
                                                                 
 gru_20 (GRU)                (None, 768, 128)          99072     
                                                                 
 gru_21 (GRU)                (None, 64)                37248     
                                                                 
 dense_12 (Dense)            (None, 2)                 130       
                                                                 
Total params: 264,450
Trainable params: 264,450
Non-trainable params: 0
_________________________________________________________________


In [203]:
with open("anomaly.pkl", "rb") as file_to_read:
    anomaly = pickle.load(file_to_read)
    
anomaly_np = np.asarray(anomaly)
anomaly_np_dum = pd.get_dummies(anomaly_np)
anomaly_np_dum

,0,1
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
...,...,...
61195,1,0
61196,0,1
61197,1,0
61198,1,0


In [204]:
from sklearn.model_selection import train_test_split
X_train_an, X_test_an, y_train_an, y_test_an = train_test_split(X, anomaly_np_dum, test_size=0.1, random_state=42)

In [205]:
history_anomaly = model_anom.fit(X_train_an, y_train_an, batch_size=32, epochs=50, validation_data=(X_test_an,y_test_an))

Epoch 1/50
1722/1722 [==============================] - 77s 44ms/step - loss: 0.2928 - accuracy: 0.9089 - val_loss: 0.1655 - val_accuracy: 0.9477
Epoch 2/50
1722/1722 [==============================] - 75s 44ms/step - loss: 0.1685 - accuracy: 0.9453 - val_loss: 0.1591 - val_accuracy: 0.9515
Epoch 3/50
1722/1722 [==============================] - 75s 44ms/step - loss: 0.1591 - accuracy: 0.9490 - val_loss: 0.1596 - val_accuracy: 0.9485
Epoch 4/50
1722/1722 [==============================] - 75s 44ms/step - loss: 0.1482 - accuracy: 0.9516 - val_loss: 0.1505 - val_accuracy: 0.9508
Epoch 5/50
1722/1722 [==============================] - 76s 44ms/step - loss: 0.1413 - accuracy: 0.9528 - val_loss: 0.1350 - val_accuracy: 0.9549
Epoch 6/50
1722/1722 [==============================] - 76s 44ms/step - loss: 0.1361 - accuracy: 0.9538 - val_loss: 0.1212 - val_accuracy: 0.9605
Epoch 7/50
1722/1722 [==============================] - 76s 44ms/step - loss: 0.1291 - accuracy: 0.9549 - val_loss: 0.1188 -

NameError: name 'models' is not defined